![GitHub Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)
![GitHub Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattene
d_small.png)

In this notebook, we'll train two classifiers to predict itemsets in the Online Retail dataset. We'll use this classic machine learning problem as a brief introduction to using Apache Spark local mode in a notebook.

In [1]:
import pandas as pd
df = pd.read_csv('Online_Retail.csv', encoding = "ISO-8859-1")
s2 =df.groupby('InvoiceNo')['StockCode'].apply(','.join)
s2 = s2.reset_index()
s2[:5].to_csv('Online_Retail.txt', columns = ["StockCode"], header=False, index=False)

Here we import pandas as our library to build our train model. The dataset being involved is 'Online_Retail.csv'. And the variables 'InvoiceNo' and 'StockCode' will serve as the variables in our train model. And the columns will be 'StockCode'.

In [2]:
from pyspark.mllib.fpm import FPGrowth
from pyspark import SparkContext
sc = SparkContext()

First we create a SparkContext, the main object in the Spark API. The FPGrowth has to be imported from the machine learning library in pyspark.

In [3]:
data = sc.textFile("Online_Retail.txt")
transactions = data.map(lambda line: line.strip().split(','))

Now we have to convert the csv file into a txt file for our dataset, then using coma as the separator.

In [4]:
model = FPGrowth.train(transactions, minSupport = 0.15, numPartitions = 10)
freqItemsets = model.freqItemsets().collect()

len(freqItemsets)

4241

We build a train model with the transaction, minSupport at 0.15 and numPartitions at 10. Then we assignment the train model to freqItemsets, and finally we use the len function to display the length of freqItemsets.

In [5]:
freqItemsetsDict = {}

for i in range(0, len(freqItemsets)):
    freqItemsetsDict[str(freqItemsets[i].items)] = freqItemsets[i].freq
    
minConfidence = .6
rules = sorted(model._java_model.generateAssociationRules(minConfidence).collect(), 
    key=lambda x: x.confidence(), reverse=True)

for j in range(0, len(rules)):
    antecedent = list(rules[j].antecedent())
    consequent = list(rules[j].consequent())
    ruleItemset = str(antecedent + consequent)
    
    try:
        support = "{0:.2f}".format(freqItemsetsDict[ruleItemset] / transactions.count())
    except KeyError:
        next
    
    confidence = "{0:.2f}".format(rules[j].confidence())
    
    try:
        print(antecedent, '=>', consequent, '(support: '+ str(support), 'confidence: '+ str(confidence) + ')')
    except NameError:
        next

['21754', '22310', '84969', '22623', '22622', '48187"'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22623', '22622', '48187"'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22623', '22622', '48187"'] => ['21755'] (support: 0.20 confidence: 1.00)
['22749'] => ['22748'] (support: 0.20 confidence: 1.00)
['22749'] => ['"84879'] (support: 0.20 confidence: 1.00)
['22749'] => ['22745'] (support: 0.20 confidence: 1.00)
['22749'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22749'] => ['21755'] (support: 0.20 confidence: 1.00)
['22749'] => ['21777'] (support: 0.20 confidence: 1.00)
['22749'] => ['22310'] (support: 0.20 confidence: 1.00)
['22749'] => ['22622'] (support: 0.20 confidence: 1.00)
['22749'] => ['22623'] (support: 0.20 confidence: 1.00)
['22749'] => ['21754'] (support: 0.20 confidence: 1.00)
['22749'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22748'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754',

['21754', '84969', '22622', '21777', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22745'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['21777'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['22749'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['22622'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '48187"', '22745'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '48187"', '22745'] =>

['22622', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['22623'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['22748'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['22622'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['22623'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '

['21754', '84969', '21777', '22749', '22748', '48187"', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '48187"', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '48187"', '"84879'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '48187"', '"84879'] => ['22622'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748', '22745', '"84879', '21755'] => ['21777'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748', '22745', '"84879', '21755'] => ['22623'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748', '22745', '"84879', '21755'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748', '22745', '"84879', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748', '22745', '"84879', '21755'] => ['84969'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748', '22745', '"84

['22310', '84969', '22623', '22749', '22745'] => ['"84879'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22749', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22749', '22745'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22749', '22745'] => ['22622'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22749', '22745'] => ['21754'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '21777', '22748'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '21777', '22748'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '21777', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '21777', '22748'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '21777', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '21777', '22748'] => ['22749'] (support: 0.20 co

...

['21754', '84969', '22622', '21777', '22748', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22748', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22748', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22748', '"84879'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622'] => ['22310'] (support: 0.20 confidence: 1.0

['21754', '22310', '84969', '21777', '22749', '22748', '48187"'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '21777', '22749', '22748', '48187"'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '21777', '22749', '22748', '48187"'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '21777', '22749', '22748', '48187"'] => ['22622'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['21777'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['22745'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['22623'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['21754'] 

['21754', '22622', '22749', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22749', '22748'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22749', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22749', '22748'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '22745', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '22745', '21755'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '22745', '21755'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '22745', '21755'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '22745', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)
['22310', '22622', '21777', '22749', '22748', '48187"', '22745', '21755'] => ['22623'] (support: 0.20 confidence: 1

['21754', '22310', '22748'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['22745'] (support: 0.20 confidence: 1.

['21754', '22622', '22748', '48187"'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['84969'] (support: 0.20 confidence: 1.00)


First we create a dictonary for the itemsets, then use the association rules with the minConfidence at 0.6 to generate the support and confidence for each itemsets.